# FFT saved file test

In [39]:
import numpy as np
import h5py as h5
from fft_helper import fft_to_dBm

%matplotlib widget

from numpy.fft import fft,rfft,rfftfreq
import matplotlib.pyplot as plt

fname = './raw_test/2022-03-17/17/RAW_output1_0.h5'

f = h5.File(fname, 'r')

# py = f['amplitude'][...]

time = f['block_time'][...]

print(f['block_time'][...])

st = 1/480.
px = rfftfreq(4096*16, d=st)

#print(py[0,0:100])

# print(time[0:20])
# plt.plot(px, fft_to_dBm(py[-1,...]))
# plt.show()


[b'2022-03-17T17:26:01.466352' b'2022-03-17T17:26:01.490921'
 b'2022-03-17T17:26:01.530483' b'2022-03-17T17:26:01.557630'
 b'2022-03-17T17:26:01.584256' b'2022-03-17T17:26:01.611098'
 b'2022-03-17T17:26:01.637648' b'2022-03-17T17:26:01.665094'
 b'2022-03-17T17:26:01.691904' b'2022-03-17T17:26:01.719405'
 b'2022-03-17T17:26:01.746212' b'2022-03-17T17:26:01.773405'
 b'2022-03-17T17:26:01.800784' b'2022-03-17T17:26:01.827730'
 b'2022-03-17T17:26:01.854781' b'2022-03-17T17:26:01.881259'
 b'2022-03-17T17:26:01.907213' b'2022-03-17T17:26:01.933852'
 b'2022-03-17T17:26:01.960289' b'2022-03-17T17:26:01.986252'
 b'2022-03-17T17:26:02.012343' b'2022-03-17T17:26:02.037946'
 b'2022-03-17T17:26:02.064168' b'2022-03-17T17:26:02.089408'
 b'2022-03-17T17:26:02.116610' b'2022-03-17T17:26:02.142142'
 b'2022-03-17T17:26:02.168922' b'2022-03-17T17:26:02.194886'
 b'2022-03-17T17:26:02.220470' b'2022-03-17T17:26:02.246960'
 b'2022-03-17T17:26:02.276317' b'2022-03-17T17:26:02.307022']


# GPU FFT test

In [31]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()

2022-03-05 16:19:11.111114: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 47169 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:7c:00.0, compute capability: 7.5


['/device:GPU:0']

In [4]:
import torch

torchdevice = 0

names =[]

name = 'PyTorch\nGPU'
device_type = 'cuda'
device = torch.device('cuda:{}'.format(torchdevice))
names.append(name)
print('{} available: {}'.format(name, torch.cuda.is_available()))

PyTorch
GPU available: True


In [16]:
import torch
import mkl_fft
def torch_ifft2_fft2(data):
    signal_ndim = 2
    data2 = torch.fft.fft(data)
    return torch.fft.ifft(data2)

def np_ifft2_fft2(data):
    data2 = mkl_fft.fft.fft2(data)
    return mkl_fft.fft.ifft2(data2)

sizes = [512, 1024, 2048, 4096] # X * X pixel arrays/tensors
names = []
totaltimes=[]
name = 'PyTorch\nGPU'
device_type = 'cuda'
device = torch.device('cuda:{}'.format(0))
names.append(name)
print('{} available: {}'.format(name, torch.cuda.is_available()))
torchtimes = []
datas = [torch.randn(size=(x,x,2), device=device) for x in sizes]
print(f'Working on device: {datas[0].device}')
for x, data in zip(sizes, datas):
    print('{} {}x{}'.format(" ".join(name.split('\n')), x, x))
    t = %timeit -o torch_ifft2_fft2(data)
    torchtimes.append(t)
totaltimes.append(torchtimes)

PyTorch
GPU available: True
Working on device: cuda:0
PyTorch GPU 512x512
26 µs ± 3.19 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
PyTorch GPU 1024x1024
172 µs ± 27.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
PyTorch GPU 2048x2048
678 µs ± 31.3 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
PyTorch GPU 4096x4096
2.7 ms ± 35.6 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
name = 'PyTorch\nCPU'
device_type = 'cpu'
device = torch.device(device_type)
names.append(name)
torchtimes = []
datas = [torch.randn((x,x,2), device=device) for x in sizes]
print(f'Working on device: {datas[0].device}')
for x, data in zip(sizes, datas):
    print('{} {}x{}'.format(" ".join(name.split('\n')), x, x))
    t = %timeit -o torch_ifft2_fft2(data)
    torchtimes.append(t)
totaltimes.append(torchtimes)

Working on device: cpu
PyTorch CPU 512x512
296 µs ± 26.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
PyTorch CPU 1024x1024
533 µs ± 1.99 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
PyTorch CPU 2048x2048
18.1 ms ± 64.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
PyTorch CPU 4096x4096
61.4 ms ± 192 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
import numpy as np

name = 'Numpy\nCPU'
names.append(name)
nptimes = []
datas = [np.random.normal(size=(x,x)).astype('complex128') for x in sizes]
for x, data in zip(sizes, datas):
    print('{} {}x{}'.format(" ".join(name.split('\n')), x, x))
    t = %timeit -o np_ifft2_fft2(data)
    nptimes.append(t)
totaltimes.append(nptimes)

Numpy CPU 512x512
9.3 ms ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Numpy CPU 1024x1024
51 ms ± 36.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Numpy CPU 2048x2048
330 ms ± 412 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
Numpy CPU 4096x4096
1.39 s ± 1.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
import cupy as cp

def cp_ifft2_fft2(data):
    data2 = cp.fft.fft2(data)
    return cp.fft.ifft2(data2)

name = 'CuPy\nGPU'
names.append(name)
cptimes = []
device = cp.cuda.Device(0)
with device:
    datas = [cp.random.normal(size=(x,x)).astype('complex128') for x in sizes]
    for x, data in zip(sizes, datas):
        print('{} {}x{}'.format(" ".join(name.split('\n')), x, x))
        t = %timeit -o cp_ifft2_fft2(data)
        cptimes.append(t)
    totaltimes.append(cptimes)

# # Clear cupy memory
# mempool = cp.get_default_memory_pool()
# del datas
# mempool.free_all_blocks()

CuPy GPU 512x512
216 µs ± 57.5 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
CuPy GPU 1024x1024
1.01 ms ± 3.05 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
CuPy GPU 2048x2048
3.89 ms ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
CuPy GPU 4096x4096
15.8 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
